<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN02/2.3-Overfitting_and_Underfitting_PyTorch.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt


In [ ]:
import sys
import pickle

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

# Print versions in a compact form
print(f"Python version: {sys.version}")
print(f"torch: {torch.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"numpy: {np.__version__}")
print(f"pandas: {pd.__version__}")


# Bias-Varianz Trade-Off

Bei allen Beispielen, die wir im vorigen Kapitel gesehen haben - Vorhersage der Stimmung in Filmrezensionen, Themenklassifizierung und Regression der Hauspreise - konnten wir feststellen, dass die Leistung unseres Modells auf den ausgeklammerten Validierungsdaten immer nach ein paar Epochen ihren Höhepunkt erreichte und dann begann 
verschlechtern würde, d. h. unser Modell würde schnell beginnen, sich an die Trainingsdaten _überanzupassen_. 

- Überanpassung (**overfitting**) kommt bei jedem einzelnen Problem des maschinellen Lernens Problem. 
- Um das maschinelle Lernen zu beherrschen, muss man lernen, mit der Überanpassung umzugehen.


Das grundlegende Problem beim maschinellen Lernen ist der **Trade-off (Kompromiss) zwischen Optimierung und Verallgemeinerung**.

- *Optimierung* bezieht sich auf den Prozess der Anpassung eines Modells, um die bestmögliche Leistung aus den Trainingsdaten zu erzielen (das "Lernen" in "maschinelles Lernen"), während sich *Generalisierung* darauf bezieht, wie gut das trainierte Modell 
auf Daten funktioniert, die es noch nie zuvor gesehen hat. Das Ziel des Maschinellen Lernens ist es natürlich, eine gute Generalisierung zu erreichen. 


Aber Sie haben keinen Einfluss auf die Generalisierung; 
Sie können das Modell nur auf der Grundlage seiner Trainingsdaten anpassen.

## Lernkurven

- Zu Beginn des Trainings sind Optimierung und Generalisierung korreliert: Je geringer der Verlust (**loss**) bei den Trainingsdaten ist, desto geringer ist der Verlust bei den Testdaten. 
- Während dies geschieht, wird Ihr Modell als _unzureichend angepasst_ bezeichnet (high bias): Es müssen noch Fortschritte gemacht werden; das Netz hat noch nicht alle relevanten Muster in den Trainingsdaten modelliert. 
- Nach einer bestimmten Anzahl von Iterationen (Epochen) mit den Trainingsdaten verbessert sich die Generalisierung nicht mehr, und die Validierungsmetriken beginnen sich zu verschlechtern: Das Modell beginnt dann, sich zu sehr anzupassen, d. h. es beginnt, Muster zu lernen die für die Trainingsdaten spezifisch sind, die aber irreführend oder irrelevant sind, wenn es um neue Daten geht.

In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN01/Bilder/BiasVarianceTradeOff.jpg"
display(Image(url=url))


- Um zu verhindern, dass ein Modell irreführende oder irrelevante Muster lernt, die in den Trainingsdaten gefunden wurden, _ist die beste Lösung , dass man mehr Trainingsdaten_ zur Verfügung stellt. Ein Modell, das mit mehr Daten trainiert wurde, kann i.A. besser verallgemeinern. 
- Wenn dies nicht mehr möglich ist, besteht die nächstbeste Lösung darin, die Menge der Informationen, die Ihr Modell speichern darf, zu modifizieren oder die Informationen, die es speichern darf, einzuschränken. 

Wenn ein Netz es sich nur leisten kann, eine kleine Anzahl von Mustern zu speichern, wird es durch den Optimierungsprozess gezwungen, sich auf die Muster zu konzentrieren, die eine bessere Chance auf eine gute Generalisierung haben.

Der Prozess der Bekämpfung der Überanpassung auf diese Weise wird _Regelmäßigkeit_ genannt. Schauen wir uns einige der gängigsten Regularisierungs und wenden wir sie in der Praxis an, um unser Filmklassifizierungsmodell aus dem vorherigen Kapitel zu verbessern.


In [ ]:
# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_words = 10000


# Load the dataset from the pickle file
with open("./Daten/imdb_dataset.pkl", "rb") as dataset_file:
    imdb_data = pickle.load(dataset_file)

# Load the word index from the pickle file
with open("./Daten/imdb_word_index.pkl", "rb") as word_index_file:
    word_index = pickle.load(word_index_file)

# Access the dataset
train_data = imdb_data["train_data"]
train_labels = imdb_data["train_labels"]
test_data = imdb_data["test_data"]
test_labels = imdb_data["test_labels"]

print(f"Loaded {len(train_data)} training samples and {len(test_data)} test samples.")
print(f"Loaded word index with {len(word_index)} words.")

print("IMDB dataset and word index saved to pickle files.")


Hinweis: In diesem Notizbuch werden wir die IMDB-Testmenge als Validierungsmenge verwenden. Das spielt in diesem Zusammenhang keine Rolle.

Bereiten wir die Daten mit dem Code aus Kapitel 3, Abschnitt 5 vor:

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0  # set specific indices of results[i] to 1s
    return results


# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)
# Our vectorized labels
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")


In [ ]:
# Convert labels to tensors
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# Create PyTorch datasets
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Combine into datasets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# Split training data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
# Example usage with PyTorch Lightning Trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class SentimentModel(pl.LightningModule):
    def __init__(self, n_hidden=16):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(10000, n_hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.output = nn.Linear(n_hidden, 1)
        self.sigmoid = nn.Sigmoid()
        self.criterion = nn.BCELoss()

        # Initialize metric storage
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return self.sigmoid(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(acc.item())
        return {"loss": loss, "train_acc": acc}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        self.val_losses.append(loss.item())
        self.val_accuracies.append(acc.item())
        return {"val_loss": loss, "val_acc": acc}

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.001)


In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=True, mode="min")

# Initialize the PyTorch Lightning Trainer
max_epochs = 30

trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    # callbacks=[early_stopping],
    log_every_n_steps=1,
)


In [ ]:
# Instantiate the model
model = SentimentModel(n_hidden=16)

# Train the model
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)


In [ ]:
# Function to plot learning curves
def plot_learning_curves(model):
    epochs_train = (
        np.array(range(1, len(model.train_losses) + 1))
        / len(model.train_losses)
        * max_epochs
    )
    epochs_val = (
        np.array(range(1, len(model.val_losses) + 1))
        / len(model.val_losses)
        * max_epochs
    )

    # Plotting
    plt.figure(figsize=(12, 5))

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_train, model.train_losses, "b.-", label="Training Loss")
    plt.plot(epochs_val, model.val_losses, "r.-", label="Validation Loss")
    plt.title("Loss Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()

    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_train, model.train_accuracies, "b.-", label="Training Accuracy")
    plt.plot(epochs_val, model.val_accuracies, "r.-", label="Validation Accuracy")
    plt.title("Accuracy Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
# Plot learning curves
plot_learning_curves(model)


# Fighting overfitting

## Verkleinerung des Netzwerks


- Die einfachste Möglichkeit, Overfitting zu verhindern, besteht darin, die Größe des Modells zu reduzieren, d. h. die Anzahl der lernbaren Parameter im Modell (die durch die Anzahl der Schichten und die Anzahl der Einheiten pro Schicht bestimmt wird). 
- Beim Deep Learning wird die Anzahl der lernbaren Parameter in einem Modell oft als **Kapazität** des Modells bezeichnet. Intuitiv betrachtet hat ein Modell mit mehr Parametern eine größere "Modellierungskapazität (höhere Dimension des Hypothesenraums" und kann daher in der Lage, eine perfekte wörterbuchähnliche Abbildung zwischen den Trainingsproben und ihren Zielen zu erlernen, eine Abbildung ohne jegliche  Generalisierungskraft. 
- Ein Modell mit 500.000 binären Parametern könnte zum Beispiel leicht die Klasse aller Ziffern in der MNIST-Trainingsmenge zu erlernen: Für jede der 50.000 Ziffern würden nur 10 binäre Parameter benötigt. Ein solches Modell wäre unbrauchbar für die Klassifizierung neuen Ziffernproben. 

**Denken Sie immer daran: Deep-Learning-Modelle sind in der Regel gut darin, sich an die Trainingsdaten anzupassen, aber die eigentliche Herausforderung ist die Verallgemeinerung, nicht die Anpassung.**




Wenn das Netzwerk andererseits nur über begrenzte Speicherressourcen verfügt, kann es diese Zuordnung nicht so leicht erlernen, so dass es zur Minimierung seiner Verluste 
Um den Verlust zu minimieren, muss es komprimierte Repräsentationen lernen, die eine Vorhersagekraft in Bezug auf die Ziele haben -- genau die Art von Repräsentationen, an denen wir interessiert sind. Gleichzeitig sollten Sie darauf achten, dass Sie Modelle verwenden, die über genügend Parameter verfügen, um nicht unterdurchschnittlich gut zu passen: Ihr Modell sollte nicht unter dem Mangel an Erinnerungsressourcen leiden. Es muss ein Kompromiss zwischen "zu viel Kapazität" und "nicht genug Kapazität" gefunden werden.

Leider gibt es keine magische Formel, um die richtige Anzahl von Schichten oder die richtige Grösse der einzelnen Schichten zu bestimmen. Sie müssen eine Reihe verschiedener Architekturen evaluieren (natürlich auf Ihrem Validierungssatz, nicht auf Ihrem Testsatz), um die richtige richtige Modellgrösse für Ihre Daten zu finden. 
- Der allgemeine Arbeitsablauf zur Ermittlung einer geeigneten Modellgrösse besteht darin, mit relativ wenigen Schichten und Parametern zu beginnen und dann die Grösse der Schichten zu erhöhen oder neue Schichten hinzuzufügen, bis Sie einen abnehmenden Ertrag in Bezug auf den Validierungsverlust.

Versuchen wir dies an unserem Netzwerk zur Klassifizierung von Filmrezensionen. Unser ursprüngliches Netzwerk sah wie folgt aus:

In [ ]:
# Instantiate the model
small_model = SentimentModel(n_hidden=3)


# Initialize the PyTorch Lightning Trainer

trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    # callbacks=[early_stopping],
    log_every_n_steps=1,
)

# Train the model
trainer.fit(small_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


In [ ]:
# Plot learning curves
plot_learning_curves(small_model)


Nun wollen wir versuchen, es durch dieses kleinere Netz zu ersetzen:


Wie Sie sehen können, beginnt das kleinere Netz später mit der Überanpassung als das Referenznetz (nach 6 Epochen statt nach 4) und seine Leistung verschlechtert sich viel langsamer, sobald es mit der Überanpassung beginnt.

Fügen wir nun zur Abwechslung zu diesem Benchmark ein Netz hinzu, das viel mehr Kapazität hat, weit mehr, als es das Problem rechtfertigen würde:

In [ ]:
# Instantiate the model
bigger_model = SentimentModel(n_hidden=512)

# Initialize the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    # callbacks=[early_stopping],
    log_every_n_steps=1,
)

# Train the model
trainer.fit(bigger_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


In [ ]:
# Plot learning curves
plot_learning_curves(bigger_model)


Hier sehen Sie, wie das grössere Netz im Vergleich zum Referenznetz abschneidet. Die Punkte sind die Validierungsverlustwerte des grösseren Netzes, und die Kreuze sind das ursprüngliche Netz.


Das grössere Netz beginnt fast sofort, nach nur einer Epoche, mit der Überanpassung, und die Überanpassung ist viel stärker. Sein Validierungsverlust ist auch stärker verrauscht.

In der Zwischenzeit sind hier die Trainingsverluste für unsere beiden Netzwerke:

Wie Sie sehen können, geht der Trainingsverlust bei einem größeren Netz sehr schnell gegen Null. Je mehr Kapazität das Netz hat, desto schneller ist es in der Lage 
es die Trainingsdaten modellieren kann (was zu einem niedrigen Trainingsverlust führt), aber desto anfälliger ist es für eine Überanpassung (was zu einer grossen Unterschied zwischen dem Trainings- und dem Validierungsverlust).

## Hinzufügen einer Gewichtsregulierung

Vielleicht kennen Sie das Prinzip von _Occam's Razor_: 
**Wenn es zwei Erklärungen für etwas gibt, ist die Erklärung, die am wahrscheinlichsten richtig ist, die die "einfachste", diejenige, die die wenigsten Annahmen enthält.**

Dies gilt auch für die Modelle, die von neuronalen Netzen gelernt werden: Bei einigen Trainingsdaten und einer Netzwerkarchitektur gibt es mehrere Sätze von Gewichtungswerten (mehrere _Modelle_), die die Daten erklären könnten, und 
bei einfacheren Modellen ist die Wahrscheinlichkeit einer Überanpassung geringer als bei komplexen Modellen.

- Ein "einfaches Modell" ist in diesem Zusammenhang ein Modell, bei dem die Verteilung der Parameterwerte eine geringere Entropie aufweist (oder ein Modell mit weniger Parametern insgesamt, wie wir im obigen Abschnitt gesehen haben). 
- Eine gängige Methode zur **Abschwächung der Überanpassung ist daher die Beschränkung der Komplexität eines Netzes**, indem man seine Gewichte zwingt, nur kleine Werte anzunehmen, wodurch die Verteilung der Gewichtungswerte "regelmässiger" wird. 
- Dies wird als Dies wird als *Gewichtsregulierung* bezeichnet und geschieht, indem zur Verlustfunktion des Netzes ein **Strafterm für Komplexität** hinzugefügt wird, der mit großen Gewichten verbunden ist. 

Diese Kosten gibt es in zwei Varianten:

- **L1-Regularisierung**, bei der die hinzugefügten Kosten proportional zum _absoluten Wert der Gewichtskoeffizienten_ sind (d.h. zu dem, was man die "L1-Norm" der Gewichte).
- **L2-Regularisierung**, bei der die zusätzlichen Kosten proportional zum _Quadrat des Wertes der Gewichtungskoeffizienten_ sind (d.h. zu dem, was als der "L2-Norm" der Gewichte). 

Die L2-Regularisierung wird im Zusammenhang mit neuronalen Netzen auch als _Gewichtsabbau_ bezeichnet. Lassen Sie sich nicht durch den anderen  Namen nicht verwirren: Gewichtsabnahme ist mathematisch gesehen genau dasselbe wie L2-Regularisierung.

In Keras wird die Gewichtsregularisierung hinzugefügt, indem _Gewichtsregularisierungsinstanzen_ als Schlüsselwortargumente an Schichten übergeben werden. Fügen wir L2-Gewichtsregulierung 
Regularisierung zu unserem Klassifizierungsnetzwerk für Filmrezensionen hinzu:

In [ ]:
# Example usage with PyTorch Lightning Trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class SentimentModel(pl.LightningModule):
    def __init__(self, n_hidden=16, l2_lambda=0.001):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(10000, n_hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.output = nn.Linear(n_hidden, 1)
        self.sigmoid = nn.Sigmoid()
        self.criterion = nn.BCELoss()

        # L2 regularization (weight_decay)
        self.l2_lambda = l2_lambda

        # Initialize metric storage
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return self.sigmoid(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(acc.item())
        return {"loss": loss, "train_acc": acc}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        self.val_losses.append(loss.item())
        self.val_accuracies.append(acc.item())
        return {"val_loss": loss, "val_acc": acc}

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.parameters(), lr=0.001, weight_decay=self.l2_lambda
        )


In [ ]:
# Instantiate the model
regularized_model = SentimentModel(n_hidden=512, l2_lambda=0.001)

# Initialize the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    # callbacks=[early_stopping],
    log_every_n_steps=1,
)

# Train the model
trainer.fit(
    regularized_model, train_dataloaders=train_loader, val_dataloaders=val_loader
)


`l2(0.001)` means that every coefficient in the weight matrix of the layer will add `0.001 * weight_coefficient_value` to the total loss of 
the network. Note that because this penalty is _only added at training time_, the loss for this network will be much higher at training 
than at test time.

Here's the impact of our L2 regularization penalty:

In [ ]:
# Plot learning curves
plot_learning_curves(regularized_model)



Wie Sie sehen können, ist das Modell mit L2-Regularisierung (Punkte) viel resistenter gegen Overfitting als das Referenzmodell (Kreuze), obwohl beide Modelle die gleiche Anzahl von Parametern haben.

Als Alternative zur L2-Regularisierung können wir eine Lasso- oder L1-Regularisierung verwenden. Diese wird einige Gewichte auf Null setzen und somit eine kompakes Netzwerk forcieren.

In [ ]:
# Example usage with PyTorch Lightning Trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class SentimentModel(pl.LightningModule):
    def __init__(self, n_hidden=16, l1_lambda=0.001, l2_lambda=0.0):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(10000, n_hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.output = nn.Linear(n_hidden, 1)
        self.sigmoid = nn.Sigmoid()
        self.criterion = nn.BCELoss()

        # L2 regularization (weight_decay)
        self.l2_lambda = l2_lambda
        self.l1_lambda = l1_lambda

        # Initialize metric storage
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return self.sigmoid(x)

    def compute_l1_loss(self):
        l1_loss = 0
        for param in self.parameters():
            if param.requires_grad:
                l1_loss += torch.sum(torch.abs(param))
        return self.l1_lambda * l1_loss

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        BCE_loss = self.criterion(y_hat, y)
        # Add L1 regularization term
        l1_loss = self.compute_l1_loss()
        loss = BCE_loss + l1_loss

        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(acc.item())
        return {"loss": loss, "train_acc": acc}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        self.val_losses.append(loss.item())
        self.val_accuracies.append(acc.item())
        return {"val_loss": loss, "val_acc": acc}

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.parameters(), lr=0.001, weight_decay=self.l2_lambda
        )


In [ ]:
# Instantiate the model
regularized_model = SentimentModel(n_hidden=512, l1_lambda=1e-5, l2_lambda=0.0)

# Initialize the PyTorch Lightning Trainer
max_epochs = 30

trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    # callbacks=[early_stopping],
    log_every_n_steps=1,
)

# Train the model
trainer.fit(
    regularized_model, train_dataloaders=train_loader, val_dataloaders=val_loader
)


In [ ]:
# Plot learning curves
plot_learning_curves(regularized_model)


## Hinzufügen von Dropout

**Dropout** ist eine der effektivsten und am häufigsten verwendeten *Regularisierungstechniken* für neuronale Netze, die von Hinton und seinen Studenten an der Universität von Toronto entwickelt wurde. Dropout wird auf eine Schicht angewandt und besteht darin, dass während des Trainings **eine Reihe von Ausgangsmerkmalen (activations) der Schicht zufällig "weggelassen" (d. h. auf Null gesetzt) werden.**

Nehmen wir an, eine bestimmte Schicht hätte normalerweise einen Vektor `[0.2, 0.5, 1.3, 0.8, 1.1]` für eine bestimmte Eingabeprobe während des Trainings geliefert; 
- nach Anwendung von Dropout wird dieser Vektor einige zufällig verteilte Nulleinträge haben, z. B. `[0, 0.5, 1.3, 0, 1.1]`. 
- Die **Dropout-Rate** ist der Anteil der Merkmale, die mit Nullen versehen werden; sie wird normalerweise zwischen 0.2 und 0.5 festgelegt. 
- Zum **Testzeitpunkt werden keine Einheiten herausgenommen**, stattdessen werden die Ausgabewerte der Schicht um einen Faktor reduziert, der der Dropout-Rate entspricht, um die Tatsache auszugleichen, dass mehr Einheiten aktiv sind als zur Trainingszeit.

Betrachten wir eine Numpy-Matrix, die die Ausgabe einer Schicht, `layer_output`, in der Form `(batch_size, features)` enthält. Zum Zeitpunkt des Trainings würden wir einen Teil der Werte in der Matrix nach dem Zufallsprinzip ausschliessen:


- Zum Testzeitpunkt würden wir die Ausgabe um die Abbruchrate herunterskalieren. Hier skalieren wir um 0.5 (weil wir zuvor die Hälfte der Einheiten fallen gelassen haben):
- Beachten Sie, dass dieser Prozess implementiert werden kann, indem beide Operationen zur Trainingszeit durchgeführt werden und die Ausgabe zur Testzeit unverändert bleibt, was in der Praxis häufig der Fall ist. was in der Praxis häufig der Fall ist:



Diese Technik mag seltsam und willkürlich erscheinen. Warum sollte dies dazu beitragen, die Überanpassung zu verringern? 
Geoffry Hinton hat gesagt, dass er unter anderem von einem Betrugspräventionsmechanismus inspiriert wurde, der von Banken verwendet wird - in seinen eigenen Worten: 

*"Ich ging zu meiner Bank. Die Kassierer wechselten ständig, und ich fragte einen von ihnen, warum. Er sagte, er wisse es nicht, aber sie würden oft versetzt. Ich dachte mir, das muss daran liegen, dass es eine Zusammenarbeit zwischen den Angestellten erfordert, um die Bank erfolgreich zu betrügen. Da wurde mir klar, dass das zufällige Entfernen einer anderen Untergruppe von Neuronen bei jedem Beispiel Verschwörungen verhindern und somit die Überanpassung reduzieren würde.*

Die Kernidee ist, dass die Einführung von Rauschen in die Ausgabewerte einer Schicht zufällige Muster aufbrechen kann, die nicht signifikant sind (was Hinton als "Verschwörungen" bezeichnet) und die sich das Netzwerk merken würde, wenn kein Rauschen vorhanden wäre. 

In PyTprcj kann man `Dropout` in ein Netzwerk über die `Dropout`-Schicht einführen, die auf die Ausgabe der Schicht direkt davor angewendet wird, z. B.:

Fügen wir zwei `Dropout`-Schichten zu unserem IMDB-Netzwerk hinzu, um zu sehen, wie gut sie die Überanpassung reduzieren:

In [ ]:
# Example usage with PyTorch Lightning Trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class SentimentModel(pl.LightningModule):
    def __init__(self, n_hidden=16, l1_lambda=0.001, l2_lambda=0.0):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(10000, n_hidden)
        self.dropout1 = nn.Dropout(0.5)  # Dropout layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.dropout2 = nn.Dropout(0.5)  # Dropout layer
        self.output = nn.Linear(n_hidden, 1)
        self.sigmoid = nn.Sigmoid()
        self.criterion = nn.BCELoss()

        # L2 regularization (weight_decay)
        self.l2_lambda = l2_lambda

        # Initialize metric storage
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.output(x)
        return self.sigmoid(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        self.train_losses.append(loss.item())
        self.train_accuracies.append(acc.item())
        return {"loss": loss, "train_acc": acc}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        self.val_losses.append(loss.item())
        self.val_accuracies.append(acc.item())
        return {"val_loss": loss, "val_acc": acc}

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.parameters(), lr=0.001, weight_decay=self.l2_lambda
        )


Schauen wir uns das Resultat an:

In [ ]:
# Instantiate the model
dropout_model = SentimentModel(n_hidden=512, l1_lambda=1e-5, l2_lambda=0.0)

# Initialize the PyTorch Lightning Trainer
max_epochs = 30

trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    # callbacks=[early_stopping],
    log_every_n_steps=1,
)

# Train the model
trainer.fit(dropout_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


In [ ]:
# Plot learning curves
plot_learning_curves(dropout_model)


## Zusammenfassung: Overfitting bekämpfen

Auch dies ist eine deutliche Verbesserung gegenüber dem Referenznetz.

Zusammenfassend lässt sich sagen, dass die gängigsten Methoden zur Vermeidung von Overfitting in neuronalen Netzen wie folgt aussehen

* Mehr Trainingsdaten erhalten.
* Verringern der Kapazität des Netzes.
* Hinzufügen einer Gewichtsregulierung.
* Hinzufügen von Dropout.